# YOUR FIRST LAB - Week 1, Day 1
## 🎯 AI-Powered Web Summarization with Ollama

### What You'll Build
An intelligent web browser that automatically summarizes websites using Large Language Models.

---

### Prerequisites
- ✅ Docker containers running (Conda + Ollama)
- ✅ LLM kernel selected (Python 3.11)
- ✅ Global `.env` file configured
- ✅ Ollama accessible at `http://localhost:11434`

### Learning Objectives

**1. Environment Setup**
- Load environment variables from `.env`
- Connect to Ollama API (OpenAI-compatible interface)

**2. Web Scraping**
- Extract website content with BeautifulSoup
- Handle HTML parsing and cleaning

**3. Prompt Engineering**
- Create effective system and user prompts
- Structure messages for LLM APIs

**4. LLM API Integration**
- Make API calls to Ollama
- Control model behavior (temperature, etc.)

**5. Practical Application**
- Build summarization function
- Generate email subject lines

**Expected Output:** A working prototype that summarizes any URL.

---

### Quick Start
1. Press `Shift + Enter` to execute each cell
2. Install dependencies (uncomment if needed)
3. Verify `.env` configuration loads
4. Run connection test
5. Experiment with different websites

---

### 💡 Learning Approach
Execute this notebook yourself after watching the lecture. Add print statements, experiment with variations, and share your work on GitHub to showcase your skills.

## 🔧 Setup

### Select the Kernel

1. Click **"Select Kernel"** (top-right)
2. Choose **`llm (Python 3.11.x)`**

### Prerequisites

- Docker containers running (`conda-jupyter`, `ollama`)
- Global `.env` configured at `/workspace/.env`
- Ollama accessible at `http://localhost:11434`

**Note:** Full setup instructions are in the [README](../README.md).

In [ ]:
# Installers
import sys
# Uncomment to install required packages
# !{sys.executable} -m pip install python-dotenv
# !{sys.executable} -m pip install beautifulsoup4
# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install openai  # openai library works with Ollama too!


## Instalación de Dependencias

Ejecuta la siguiente celda para instalar las dependencias necesarias. 

**Nota importante:** Estas se instalarán en el entorno correcto (LLM) gracias al uso de `sys.executable`.

# Connecting to OpenAI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  

## Troubleshooting if you have problems:

If you get a "Name Error" - have you run all cells from the top down? Head over to the Python Foundations guide for a bulletproof way to find and fix all Name Errors.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [ ]:
# Initialize OpenAI client pointing to Ollama
openai = OpenAI(
    base_url=f"{ollama_base_url}/v1",  # Ollama exposes OpenAI-compatible API at /v1
    api_key=ollama_api_key  # Using API key from .env
)

# Test message
message = "Hello! This is my first message to you via Ollama! Hi!"

messages = [{"role": "user", "content": message}]

messages


In [ ]:
# Load environment variables from global .env file

# Path to global .env file (accessible from Docker container)
# D:\dockerVolumes\conda\notebooks is mounted as /workspace in the container
global_env_path = '/workspace/.env'

# Load the configuration
load_dotenv(dotenv_path=global_env_path, override=True)

# Get Ollama configuration from .env
ollama_base_url = os.getenv('OLLAMA_BASE_URL')
ollama_api_key = os.getenv('OLLAMA_API_KEY')
ollama_model = os.getenv('OLLAMA_MODEL')

# Check the configuration
if not ollama_base_url:
    print("❌ Error: OLLAMA_BASE_URL not found in .env file!")
    print(f"   Looking for .env at: {global_env_path}")
elif not ollama_api_key:
    print("❌ Error: OLLAMA_API_KEY not found in .env file!")
elif not ollama_model:
    print("❌ Error: OLLAMA_MODEL not found in .env file!")
else:
    print("✅ Ollama configuration loaded successfully!")
    print(f"   Base URL: {ollama_base_url}")
    print(f"   Model: {ollama_model}")
    print(f"   API Key: {'*' * 40}{ollama_api_key[-8:]}")  # Hide most of the key
    print(f"   Loaded from: {global_env_path}")


❌ Error: OLLAMA_BASE_URL not found in .env file!
   Looking for .env at: D:/dockerInfraProjects/conda/.env


In [29]:
# Make the API call to Ollama using the configured model
response = openai.chat.completions.create(
    model=ollama_model,  # Using the model from .env
    messages=messages
)
response.choices[0].message.content


'Hello! 👋 Great to meet you—welcome to Ollama! How can I help you today?'

### 🔍 Alternative: Native Ollama API Call (Without OpenAI Client)

The cell above uses the **OpenAI client library** pointing to Ollama. Here's how you would call Ollama **directly** using its native API with `requests`:

**Pros of direct approach:**
- ✅ No dependency on OpenAI library
- ✅ Direct control over HTTP requests
- ✅ Explicit about using Ollama

**Cons:**
- ❌ More verbose code
- ❌ Need to handle HTTP errors manually
- ❌ Less portable (can't switch to OpenAI easily)

In [31]:
# Let's try out this utility

ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Conne

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [32]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarkyassistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [33]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [34]:
# Simple test with Ollama
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = openai.chat.completions.create(
    model=ollama_model,
    messages=messages
)
response.choices[0].message.content


'4'

## And now let's build useful messages for GPT-4.1-mini, using a function

In [35]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [36]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarkyassistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nHome - Edward Donner\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. 

## Time to bring it together - the API for OpenAI is very simple!

In [37]:
# Call Ollama API to summarize website content

def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model=ollama_model,  # Using Ollama model from .env
        messages=messages_for(website)
    )
    return response.choices[0].message.content


In [38]:
summarize("https://edwarddonner.com")

'### Edward Donner’s Personal Hub  \n> *Code, LLMs, a dead‑beat DJ, and a whole lot of “I can’t possibly know what I’m talking about.”*\n\n- **Who’s this?**  \n  - Ed is the “co‑founder & CTO” of Nebula.io, a startup turning AI into a talent‑matching matchmaker.  \n  - He’s the former CEO of untapt (yes, the one that got sold in 2021).  \n  - When he’s not writing code and feeding data into models, he’s pretending he can still drop a track at a club.\n\n- **What else lives here?**  \n  - *Connect\u202fFour* – probably a literal game‑board app, but maybe it’s a metaphor for the inevitable “I win, you lose” in the tech world.  \n  - *Outsmart* – an arena for LLMs to duel in diplomacy and deviousness; basically a “who gets less offended” showdown.\n\n- **Recent “news” (if you want to feel the beat of 2025):**  \n  1. **Sept\u202f15\u202f2025:** “AI in Production: Gen\u202fAI & Agentic\u202fAI on AWS at scale” – because scaling AI is still a mystery worthy of your calendar.  \n  2. **May\u

In [39]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [40]:
display_summary("https://edwarddonner.com")

# Edward Donner’s One‑Page Showboat

- **About the man**  
  “Hi, I’m Ed.”—coding whiz, DJ‑in‑the‑making, and an *expert* in half‑understood Hacker News topics. A co‑founder/CTO of Nebula.io (yes, that’s a real company), he claims patents, press, and a *happy* customer base—all while still having time to play with LLMs.

- **What’s actually on the page**  
  * **Connect Four & Outsmart** – a fancy arena where LLMs supposedly duel in diplomacy and deviousness; looks more like a placeholder than a finished product.  
  * **Posts (2025 highlights)** –  
    * “AI in Production: Gen AI and Agentic AI on AWS at scale” (September 15)  
    * “Connecting my courses – become an LLM expert and leader” (May 28)  
    * “2025 AI Executive Briefing” (May 18)  
    * “The Complete Agentic AI Engineering Course” (April 21)  
    (All sound impressive… until you read the actual blog link, if it even exists.)

- **Contact & socials**  
  * Email: *ed@edwarddonner.com* (just because it feels official).  
  * LinkedIn, Twitter, Facebook – because every founder needs a multi‑channel presence.

In short: a self‑promoting CV wrapped in a single page with a few oddly named projects and a handful of dated announcements that pretend to be cutting edge.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [41]:
display_summary("https://cnn.com")

**CNN’s “Breaking News, Latest News and Videos” page**  
> A wall of menus and ad‑feedback boxes so dense you’d think it’s a paper‑back novel.  
> 
> - **Header**: the usual “US – World – Politics – Business” carousel of tabs, as if CNN were trying to juggle a thousand categories in a single swoop.  
> - **Ad‑Feedback Form**: “How relevant is this ad to you?” – because obviously that’s the most newsworthy thing you can do today.  
> - **Content**: Nothing except navigation and an endless list of sub‑sections: from “Ukraine‑Russia War” to “Games” to “Weather.” No actual stories, just a labyrinth of links.  
> - **Live TV & Audio**: Promises of “Watch,” “Listen,” and “Live TV,” but no live stream actually visible here.  
> - **International Editions**: Spanish, Arabic, etc., all standing in front of a blank page.  

No breaking news, no announcements—just a beautifully cluttered interface that makes you wonder if you’re on CNN or a corporate website for a hyper‑specific product line.

In [42]:
display_summary("https://anthropic.com")

# A One‑Page *Anthropic* Quick‑Peek

- **Who is this?**  The *public‑benefit* AI think‑tank that vows to make GPT‑style chatbots *safe* and *super‑human*‑friendly.  
- **What’s the latest gossip?**  
  - **Claude Sonnet 4.5** – they’re bragging it’s the “best model in the world for agents, coding, and computer use.”  
  - **Claude Haiku 4.5** – claimed to manage *context* on their “Developer Platform” (so you can keep track of 512‑token conversations).  
- **Other model names** you’ll see floating around: Opus, Sonnet, Haiku—no surprise, this is an a‑to‑z alphabet of AI.  
- **Tone & Ethics**: They sprinkle slogans like “AI will have a vast impact … we build AI to serve humanity’s long‑term well‑being” and mention a “trust center.”  Sound a bit… philosophical?  
- **Navigation (ignored)**: All that “Try Claude… Log in… Download app” is just site meat.  

In short: Anthropic is the safety‑first, ethically‑charged, AI‑overlords’ playground, releasing shiny new Claude models while patting themselves on the back for being the benevolent force in the silicon wars.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [ ]:
# EJEMPLO: Generador de líneas de asunto para emails

# Step 1: Define el system prompt - Instrucciones para el modelo
email_system_prompt = """
You are a professional email assistant. 
Analyze the email content provided and suggest a clear, concise, and professional subject line.
The subject line should be under 60 characters and capture the main purpose of the email.
Respond with ONLY the subject line, nothing else.
"""

# Step 2: Define el contenido del email de ejemplo
sample_email = """
Hi John,

I hope this email finds you well. I wanted to reach out regarding our upcoming 
quarterly review meeting scheduled for next Monday. We need to discuss the 
Q4 sales performance, review the new marketing strategy, and set targets 
for Q1 2026.

Could you please prepare the sales reports and bring the budget proposals? 
Also, please confirm if 2 PM works for you, or if we need to reschedule.

Looking forward to hearing from you.

Best regards,
Sarah
"""

# Step 3: Crea la lista de mensajes en el formato que espera la API
email_messages = [
    {"role": "system", "content": email_system_prompt},
    {"role": "user", "content": f"Email content:\n\n{sample_email}"}
]

# Step 4: Llama a Ollama para generar la línea de asunto
response = openai.chat.completions.create(
    model=ollama_model,
    messages=email_messages,
    temperature=0.7  # Añadimos temperatura para controlar la creatividad
)

# Step 5: Obtén y muestra el resultado
suggested_subject = response.choices[0].message.content
print("📧 Email Original:")
print("-" * 60)
print(sample_email)
print("\n" + "=" * 60)
print(f"✨ Línea de Asunto Sugerida: {suggested_subject}")
print("=" * 60)

In [43]:
# EJEMPLO: Generador de líneas de asunto para emails

# Step 1: Define el system prompt - Instrucciones para el modelo
email_system_prompt = """
You are a professional email assistant. 
Analyze the email content provided and suggest a clear, concise, and professional subject line.
The subject line should be under 60 characters and capture the main purpose of the email.
Respond with ONLY the subject line, nothing else.
"""

# Step 2: Define el contenido del email de ejemplo
sample_email = """
Hi John,

I hope this email finds you well. I wanted to reach out regarding our upcoming 
quarterly review meeting scheduled for next Monday. We need to discuss the 
Q4 sales performance, review the new marketing strategy, and set targets 
for Q1 2026.

Could you please prepare the sales reports and bring the budget proposals? 
Also, please confirm if 2 PM works for you, or if we need to reschedule.

Looking forward to hearing from you.

Best regards,
Sarah
"""

# Step 3: Crea la lista de mensajes en el formato que espera la API
email_messages = [
    {"role": "system", "content": email_system_prompt},
    {"role": "user", "content": f"Email content:\n\n{sample_email}"}
]

# Step 4: Llama a Ollama para generar la línea de asunto
response = openai.chat.completions.create(
    model=ollama_model,
    messages=email_messages,
    temperature=0.7  # Añadimos temperatura para controlar la creatividad
)

# Step 5: Obtén y muestra el resultado
suggested_subject = response.choices[0].message.content
print(" Email Original:")
print("-" * 60)
print(sample_email)
print("\n" + "=" * 60)
print(f" Línea de Asunto Sugerida: {suggested_subject}")
print("=" * 60)

 Email Original:
------------------------------------------------------------

Hi John,

I hope this email finds you well. I wanted to reach out regarding our upcoming 
quarterly review meeting scheduled for next Monday. We need to discuss the 
Q4 sales performance, review the new marketing strategy, and set targets 
for Q1 2026.

Could you please prepare the sales reports and bring the budget proposals? 
Also, please confirm if 2 PM works for you, or if we need to reschedule.

Looking forward to hearing from you.

Best regards,
Sarah


 Línea de Asunto Sugerida: Q4 Review Meeting Prep: Sales Reports & Budget Proposal
